In [1]:
!git clone https://huggingface.co/datasets/PaulLerner/viquae_all_images

Cloning into 'viquae_all_images'...
remote: Enumerating objects: 38, done.
remote: Total 38 (delta 0), reused 0 (delta 0), pack-reused 38
Unpacking objects: 100% (38/38), 4.79 KiB | 140.00 KiB/s, done.


In [2]:
!git clone https://huggingface.co/datasets/PaulLerner/viquae_images

Cloning into 'viquae_images'...
remote: Enumerating objects: 15, done.
remote: Total 15 (delta 0), reused 0 (delta 0), pack-reused 15
Unpacking objects: 100% (15/15), 1.61 KiB | 110.00 KiB/s, done.


In [3]:
!git clone https://huggingface.co/datasets/PaulLerner/viquae_dataset

Cloning into 'viquae_dataset'...
remote: Enumerating objects: 18, done.
remote: Total 18 (delta 0), reused 0 (delta 0), pack-reused 18
Unpacking objects: 100% (18/18), 2.10 KiB | 89.00 KiB/s, done.


In [4]:
!git clone https://huggingface.co/datasets/PaulLerner/viquae_wikipedia

Cloning into 'viquae_wikipedia'...
remote: Enumerating objects: 20, done.
remote: Total 20 (delta 0), reused 0 (delta 0), pack-reused 20
Unpacking objects: 100% (20/20), 2.40 KiB | 94.00 KiB/s, done.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("PaulLerner/viquae_v4-alpha_passages")

In [5]:
from datasets import load_dataset
dataset = load_dataset('PaulLerner/viquae_dataset')
dataset

Found cached dataset json (/Users/saban.akay/.cache/huggingface/datasets/PaulLerner___json/PaulLerner--viquae_dataset-cbf702af5ea079d1/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'input', 'kilt_id', 'id', 'meta', 'original_question', 'output', 'url', 'wikidata_id'],
        num_rows: 1190
    })
    validation: Dataset({
        features: ['image', 'input', 'kilt_id', 'id', 'meta', 'original_question', 'output', 'url', 'wikidata_id'],
        num_rows: 1250
    })
    test: Dataset({
        features: ['image', 'input', 'kilt_id', 'id', 'meta', 'original_question', 'output', 'url', 'wikidata_id'],
        num_rows: 1257
    })
})

In [6]:
item = dataset['test'][0]

In [7]:
type(item)

dict

In [8]:
item['url']

'http://upload.wikimedia.org/wikipedia/commons/thumb/a/ae/Jackie_Wilson.png/512px-Jackie_Wilson.png'

In [9]:
item['image']

'512px-Jackie_Wilson.png'

In [33]:
#from meerqat.data.loading import load_image_batch
#image = load_image_batch([item['image']])[0]
#type(image), image.size

In [11]:
item['input']

"this singer's re-issued song became the UK Christmas number one after helping to advertise what brand?"

In [12]:
item['output']['original_answer']

"Levi's"

In [13]:
data_files = dict(
    humans_with_faces='humans_with_faces.jsonl.gz',
    humans_without_faces='humans_without_faces.jsonl.gz',
    non_humans='non_humans.jsonl.gz'
)
kb = load_dataset('PaulLerner/viquae_wikipedia', data_files=data_files)
kb

Found cached dataset json (/Users/saban.akay/.cache/huggingface/datasets/PaulLerner___json/PaulLerner--viquae_wikipedia-b80f4d3edaea2505/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    humans_with_faces: Dataset({
        features: ['anchors', 'categories', 'image', 'kilt_id', 'text', 'url', 'wikidata_info', 'wikipedia_id', 'wikipedia_title'],
        num_rows: 506237
    })
    humans_without_faces: Dataset({
        features: ['anchors', 'categories', 'image', 'kilt_id', 'text', 'url', 'wikidata_info', 'wikipedia_id', 'wikipedia_title'],
        num_rows: 35736
    })
    non_humans: Dataset({
        features: ['anchors', 'categories', 'image', 'kilt_id', 'text', 'url', 'wikidata_info', 'wikipedia_id', 'wikipedia_title'],
        num_rows: 953379
    })
})

In [14]:
item = kb['humans_with_faces'][0]
item['wikidata_info']['wikidata_id'], item['wikidata_info']['wikipedia_title']

('Q313590', 'Alain Connes')

In [15]:
item['image']

'512px-Alain_Connes.jpg'

In [16]:
# the text is stored in a list of string, one per paragraph
type(item['text']['paragraph']), len(item['text']['paragraph'])

(list, 25)

In [17]:
# the text is stored in a list of string, one per paragraph
item['text']['paragraph'][1]

'Alain Connes (; born 1 April 1947) is a French mathematician, currently Professor at the Collège de France, IHÉS, Ohio State University and Vanderbilt University. He was an Invited Professor at the Conservatoire national des arts et métiers (2000).\n'

In [18]:
#to concatenate these three datasets to get a single dataset (e.g. to split the articles in passages)
from datasets import concatenate_datasets
kb['humans_with_faces'] = kb['humans_with_faces'].map(lambda item: {'is_human': True})
kb['humans_without_faces'] = kb['humans_without_faces'].map(lambda item: {'is_human': True})
kb['non_humans'] = kb['non_humans'].map(lambda item: {'is_human': False})
kb_recat = concatenate_datasets([kb['non_humans'], kb['humans_with_faces'], kb['humans_without_faces']])
kb_recat.save_to_disk('data/viquae_wikipedia_recat')

Loading cached processed dataset at /Users/saban.akay/.cache/huggingface/datasets/PaulLerner___json/PaulLerner--viquae_wikipedia-b80f4d3edaea2505/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-ce070adf6f3013e6.arrow
Loading cached processed dataset at /Users/saban.akay/.cache/huggingface/datasets/PaulLerner___json/PaulLerner--viquae_wikipedia-b80f4d3edaea2505/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-5ebd487f849e3d9d.arrow
Loading cached processed dataset at /Users/saban.akay/.cache/huggingface/datasets/PaulLerner___json/PaulLerner--viquae_wikipedia-b80f4d3edaea2505/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-47d688b45c84df05.arrow


Saving the dataset (0/22 shards):   0%|          | 0/1495352 [00:00<?, ? examples/s]

In [29]:
!python -m meerqat.data.loading passages data/viquae_wikipedia_recat data/viquae_passages experiments/passages/config.json --disable_caching

/Users/saban.akay/Desktop/TU/3_ws_23/EDDS/edds/ViQuAE/meerqat/data/loading.py:458: FutureWarning: set_caching_enabled is deprecated and will be removed in the next major version of datasets. Use datasets.enable_caching() or datasets.disable_caching() instead. This function will be removed in a future version of datasets.
  set_caching_enabled(not args['--disable_caching'])
Map: 100%|████████████████████| 1495352/1495352 [41:34<00:00, 599.51 examples/s]


In [30]:
!python -m meerqat.data.loading map data/viquae_wikipedia_recat wikipedia_title title2index.json --inverse --disable_caching

/Users/saban.akay/Desktop/TU/3_ws_23/EDDS/edds/ViQuAE/meerqat/data/loading.py:458: FutureWarning: set_caching_enabled is deprecated and will be removed in the next major version of datasets. Use datasets.enable_caching() or datasets.disable_caching() instead. This function will be removed in a future version of datasets.
  set_caching_enabled(not args['--disable_caching'])
Map: 100%|█████████████████| 1495352/1495352 [00:13<00:00, 108224.37 examples/s]


In [35]:
!python -m meerqat.data.loading map data/viquae_wikipedia_recat passage_index article2passage.json --disable_caching

/Users/saban.akay/Desktop/TU/3_ws_23/EDDS/edds/ViQuAE/meerqat/data/loading.py:458: FutureWarning: set_caching_enabled is deprecated and will be removed in the next major version of datasets. Use datasets.enable_caching() or datasets.disable_caching() instead. This function will be removed in a future version of datasets.
  set_caching_enabled(not args['--disable_caching'])


In [36]:
!python -m meerqat.ir.metrics relevant data/viquae_dataset data/viquae_passages data/viquae_wikipedia_recat/title2index.json data/viquae_wikipedia_recat/article2passage.json --disable_caching

Traceback (most recent call last):
  File "/Users/saban.akay/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/saban.akay/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/saban.akay/Desktop/TU/3_ws_23/EDDS/edds/ViQuAE/meerqat/ir/metrics.py", line 394, in <module>
    passages = load_from_disk(args['<passages>'])
  File "/Users/saban.akay/anaconda3/lib/python3.10/site-packages/datasets/load.py", line 2626, in load_from_disk
    raise FileNotFoundError(f"Directory {dataset_path} not found")
FileNotFoundError: Directory data/viquae_passages not found


In [ ]:
from datasets import load_from_disk, set_caching_enabled
from meerqat.ir.metrics import find_relevant
from ranx import Run

set_caching_enabled(False)
kb = load_from_disk('data/viquae_passages/')
dataset = load_from_disk('data/viquae_dataset/train')
# to reproduce the results of the papers:
# - use DPR+Image as IR to train the reader or fine-tune ECA/ILF
# - use BM25 as IR to train DPR (then save in 'BM25_provenance_indices'/'BM25_irrelevant_indices')
run = Run.from_file('/path/to/bm25/or/multimodal_ir_train.trec')

def keep_relevant_search_wrt_original_in_priority(item, kb):
    indices = list(map(int, run[item['id']]))
    relevant_indices, _ = find_relevant(indices, item['output']['original_answer'], [], kb)
    if relevant_indices:
        item['search_provenance_indices'] = relevant_indices
    else:
        item['search_provenance_indices'] = item['original_answer_provenance_indices']
    item['search_irrelevant_indices'] = list(set(indices) - set(relevant_indices))
    return item

dataset = dataset.map(keep_relevant_search_wrt_original_in_priority, fn_kwargs=dict(kb=kb))
dataset.save_to_disk('data/viquae_dataset/train')

In [ ]:
# embed dataset images with ImageNet-ResNet50
!python -m meerqat.image.embedding data/viquae_dataset experiments/image_embedding/imagenet/config.json --disable_caching

In [ ]:
# embed KB images with ImageNet-ResNet50
!python -m meerqat.image.embedding data/viquae_wikipedia experiments/image_embedding/imagenet/config.json --disable_caching

In [ ]:
# embed dataset images with CLIP-ResNet50
!python -m meerqat.image.embedding data/viquae_dataset experiments/image_embedding/clip/config.json --disable_caching

In [ ]:
# embed KB images with CLIP-ResNet50
!python -m meerqat.image.embedding data/viquae_wikipedia experiments/image_embedding/clip/config.json --disable_caching

In [ ]:
# embed dataset images with CLIP-ViT
!python -m meerqat.image.embedding data/viquae_dataset experiments/image_embedding/clip/vit_config.json --disable_caching

In [ ]:
# embed KB images with CLIP-ViT
!python -m meerqat.image.embedding data/viquae_wikipedia experiments/image_embedding/clip/vit_config.json --disable_caching

In [ ]:
# text embedding
!python -m meerqat.ir.embedding data/viquae_wikipedia experiments/ir/viquae/clip/config.json

In [ ]:
# face detection
!python -m meerqat.image.face_detection data/viquae_dataset --disable_caching --batch_size=256

In [ ]:
# face detection
!python -m meerqat.image.face_detection data/viquae_wikipedia/humans --disable_caching --batch_size=256

In [ ]:
!git clone https://github.com/PaulLerner/insightface.git
!cd insightface
!git checkout chore/arcface_torch
!cd recognition
!pip install -e .

In [ ]:
# run
!python -m meerqat.image.face_recognition data/viquae_dataset experiments/face_recognition/config.json --disable_caching

In [ ]:
# run
!python -m meerqat.image.face_recognition data/viquae_wikipedia/humans_with_faces experiments/face_recognition/config.json --disable_caching

In [ ]:
d = load_from_disk('data/viquae_dataset')
d = d.map(lambda item: {'first_face_embedding': item['face_embedding'][0] if item['face_embedding'] is not None else None})
d.save_to_disk('data/viquae_dataset')

In [ ]:
!python -m meerqat.train.trainer fit --config=experiments/dpr/triviaqa/config.yaml